## Домашнее задание № 2
Бейзлайны и детерминированные алгоритмы item-item

weighted_random_recommendation


Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?


In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [97]:
data = pd.read_csv('/Users/ekaterina/Desktop/LEARN/IT/Рекомендательные системы/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [98]:
data['week_no'].nunique()

95

In [99]:
data.shape

(2396804, 12)

In [100]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [101]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [102]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [103]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [104]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [105]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [106]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [107]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [108]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [109]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 3.05 s, sys: 14.2 ms, total: 3.07 s
Wall time: 3.08 s


In [110]:
items

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393])

In [111]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[822944, 9249539, 1524042, 8156368, 13189735]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7122603, 6632319, 976100, 12351729, 1754607]"


### 1.2 Popularity-based recommendation

In [112]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [113]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 133 ms, sys: 31.3 ms, total: 164 ms
Wall time: 178 ms


In [114]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[822944, 9249539, 1524042, 8156368, 13189735]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7122603, 6632319, 976100, 12351729, 1754607]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [115]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [116]:
%%time

# your_code

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [117]:
# Функция, создающая датафрейм items_weights

def items_weights_creator(df):
    
    # Создаем датафрейм, который будет содержать наименование item и количество его покупок
    sales_count = df.groupby('item_id')['quantity'].count().reset_index().sort_values('quantity', ascending=False)
    
    # Переименуем столбец 'user_id'
    sales_count.rename(columns = {'quantity':'n_sold'}, inplace= True)
    
    # добавим столбец weight
    sales_count['weight'] = np.log(sales_count['n_sold'])/sum(np.log(sales_count['n_sold']))
    
    #удалим промежуточный вспомогательный столбец sales_quantity
    sales_count.drop('n_sold', axis = 1, inplace = True)
    
    items_weights = sales_count
    
    return items_weights
    

In [118]:
items_weights_train = items_weights_creator(data_train)

In [119]:
def weighted_random_recommendation(df, n=5):
    
    # добавим дополнительный параметр p, определяющий вероятность элемента оказаться в выборке choice
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [120]:
items_weights = items_weights_creator(data_train)

In [121]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x:\
                                                                   weighted_random_recommendation(items_weights, n=5))

CPU times: user 1.82 s, sys: 70.7 ms, total: 1.89 s
Wall time: 1.9 s


In [122]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[822944, 9249539, 1524042, 8156368, 13189735]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9655737, 5569186, 946541, 984905, 8249140]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7122603, 6632319, 976100, 12351729, 1754607]","[6534178, 6533889, 1029743, 6534166, 1082185]","[922636, 13776705, 897159, 5568720, 15452501]"


In [123]:
result['weighted_random_recommendation'].head(5)

0      [9655737, 5569186, 946541, 984905, 8249140]
1    [922636, 13776705, 897159, 5568720, 15452501]
2    [10355629, 1096643, 1119632, 1124444, 917178]
3     [980375, 955643, 1087916, 9245512, 12351966]
4     [1033690, 1048962, 12188131, 875027, 859154]
Name: weighted_random_recommendation, dtype: object

### Выводы по бейзлайнам
- Фиксируют базовое качество;
- Бейзлайны могут быть фильтрами;
- Иногда бейзлайны лучше ML-модели

#### Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?

In [124]:
#Рассчитаем популярность товаров на основе количества их покупок
popular = data_train.groupby('item_id')['quantity'].sum().reset_index()
popular.rename(columns={'quantity': 'n_sold'}, inplace=True)
popular_5000 = popular.sort_values('n_sold').head(5000).item_id.tolist()

Рассчитаем байзлайн RandomRecommendation на топ-5000 товаров

In [125]:
# RandomRecommender

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(popular_5000, n=5))

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[822944, 9249539, 1524042, 8156368, 13189735]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9655737, 5569186, 946541, 984905, 8249140]","[5657510, 9803285, 6919565, 9527245, 6904514]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7122603, 6632319, 976100, 12351729, 1754607]","[6534178, 6533889, 1029743, 6534166, 1082185]","[922636, 13776705, 897159, 5568720, 15452501]","[6378948, 956742, 977414, 987088, 6554163]"
